# Wildfire

In [95]:
import sys

# required 3.12 version
if [sys.version_info[i] for i in range(3)][:2] != [3, 12]: 
    raise Exception(f"Python 3.12 is required (Current is {[sys.version_info[i] for i in range(3)]})")

import os
import time
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import imutils
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [96]:
import torch

PROGRAM_NAME: str = "Wildfire"
CATEGORIES: list[str] = ["fire", "nofire"]
DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"

Dataset 1: `jafar_2023`
> Ibn Jafar, Anam; Islam, Al Mohimanul ; Binta Masud, Fatiha; Ullah, Jeath Rahmat; Ahmed, Md. Rayhan (2023), “FlameVision : A new dataset for wildfire classification and detection using aerial imagery ”, Mendeley Data, V4. https://doi.org/10.17632/fgvscdjsmt.4

Dataset 2: `madafri_2023`
> El-Madafri I, Peña M, Olmedo-Torre N. The Wildfire Dataset: Enhancing Deep Learning-Based Forest Fire Detection with a Diverse Evolving Open-Source Dataset Focused on Data Representativeness and a Novel Multi-Task Learning Approach. Forests. 2023; 14(9):1697. https://doi.org/10.3390/f14091697

In [97]:
import os
from pathlib import Path

#####################################################################
#                                                                   #
#  When you download both datasets:                                 #
#    1. Renamed them `jafar_2023` and `madafri_2023` respectively.  #
#    2. Keep the file structure unmodified.                         #
#                                                                   #
#####################################################################

'''
Wildfire
├───wildfire.ipynb
└───data
    ├───jafar_2023
    │   ├───Classification
    │   │   ├───test
    │   │   │   ├───fire
    │   │   │   └───nofire
    │   │   ├───train
    │   │   │   ├───fire
    │   │   │   └───nofire
    │   │   └───valid
    │   │       ├───fire
    │   │       └───nofire
    │   └───Detection
    │       ├───test
    │       │   ├───annotations
    │       │   └───images
    │       ├───train
    │       │   ├───annotations
    │       │   └───images
    │       └───valid
    │           ├───annotations
    │           └───images
    └───madafri_2023
        ├───test
        │   ├───fire
        │   └───nofire
        ├───train
        │   ├───fire
        │   └───nofire
        └───val
            ├───fire
            └───nofire
'''

__file__ = os.getcwd()
BASE_DIR = Path(__file__).resolve().parent / PROGRAM_NAME
DATA_DIR: Path = BASE_DIR / "data"

class ImageDataPaths1():
    def __init__(self) -> None:
        self.DIR: Path = DATA_DIR / "jafar_2023"
        self.CLASSIFICATION_DIR: Path = self.DIR / "Classification"
        self.CLASSIFICATION_TRAIN_DIR: Path = self.CLASSIFICATION_DIR / "train"
        self.CLASSIFICATION_TEST_DIR: Path = self.CLASSIFICATION_DIR / "test"
        self.CLASSIFICATION_VALID_DIR: Path = self.CLASSIFICATION_DIR / "valid"
        self.DETECTION_DIR: Path = self.DIR / "Detection"
        self.DETECTION_TRAIN_DIR: Path = self.DETECTION_DIR / "train"
        self.DETECTION_TEST_DIR: Path = self.DETECTION_DIR / "test"
        self.DETECTION_VALID_DIR: Path = self.DETECTION_DIR / "valid"

class ImageDataPaths2():
    def __init__(self) -> None:
        self.DIR: Path = DATA_DIR / "madafri_2023"
        self.TRAIN_DIR: Path = self.DIR / "train"
        self.TEST_DIR: Path = self.DIR / "test"
        self.VALID_DIR: Path = self.DIR / "val"

In [98]:
def short_path_name(path: Path) -> str:
    parts = path.parts
    return '.../' + '/'.join(parts[-4:]) if len(parts) > 4 else str(path)

In [99]:
from pandas import DataFrame

def generate_dataframe(folder_path: Path) -> DataFrame:
    print(f"[{PROGRAM_NAME}]: Generating new dataframe from '{short_path_name(folder_path)}'")
    dataframe: DataFrame = DataFrame([], columns = ["Image", "Class"])
    
    FIRE_DIR: Path = folder_path / "fire"
    NOFIRE_DIR: Path = folder_path / "nofire"
    
    fire_images: list[tuple[str, str]] = [(img.name, "fire") for img in FIRE_DIR.iterdir() if img.is_file()]
    nofire_images: list[tuple[str, str]] = [(img.name, "nofire") for img in NOFIRE_DIR.iterdir() if img.is_file()]

    data: list[tuple[str, str]] = fire_images + nofire_images
    dataframe: DataFrame = DataFrame(data, columns=["Image", "Class"])

    return dataframe

In [100]:
data_paths_1: ImageDataPaths1 = ImageDataPaths1()
data_paths_2: ImageDataPaths2 = ImageDataPaths2()

dataframe_train_partial_1: DataFrame = generate_dataframe(data_paths_1.CLASSIFICATION_TRAIN_DIR)
dataframe_test_partial_1: DataFrame = generate_dataframe(data_paths_1.CLASSIFICATION_TEST_DIR)
dataframe_valid_partial_1: DataFrame = generate_dataframe(data_paths_1.CLASSIFICATION_VALID_DIR)

dataframe_train_partial_2: DataFrame = generate_dataframe(data_paths_2.TRAIN_DIR)
dataframe_test_partial_2: DataFrame = generate_dataframe(data_paths_2.TEST_DIR)
dataframe_valid_partial_2: DataFrame = generate_dataframe(data_paths_2.VALID_DIR)

[Wildfire]: Generating new dataframe from '.../data/jafar_2023/Classification/train'
[Wildfire]: Generating new dataframe from '.../data/jafar_2023/Classification/test'
[Wildfire]: Generating new dataframe from '.../data/jafar_2023/Classification/valid'
[Wildfire]: Generating new dataframe from '.../Wildfire/data/madafri_2023/train'
[Wildfire]: Generating new dataframe from '.../Wildfire/data/madafri_2023/test'
[Wildfire]: Generating new dataframe from '.../Wildfire/data/madafri_2023/val'


In [107]:
import pandas

dataframe_train = pandas.concat([dataframe_train_partial_1, dataframe_train_partial_2], ignore_index=True)
dataframe_test = pandas.concat([dataframe_test_partial_1, dataframe_test_partial_2], ignore_index=True)
dataframe_valid = pandas.concat([dataframe_valid_partial_1, dataframe_valid_partial_2], ignore_index=True)

print(dataframe_train)

                                     Image   Class
0                             fire (1).png    fire
1                            fire (10).png    fire
2                           fire (100).png    fire
3                          fire (1000).png    fire
4                          fire (1001).png    fire
...                                    ...     ...
8682    zetong-li-eDFJt7M-1Ww-unsplash.jpg  nofire
8683    zetong-li-o0aRmm57u6s-unsplash.jpg  nofire
8684  zhidong-cao-HNh1W_GDruA-unsplash.jpg  nofire
8685    zhihao-ge-xyzLxDG5elM-unsplash.jpg  nofire
8686   zhuo-zhang-syzAO3dxJnA-unsplash.jpg  nofire

[8687 rows x 2 columns]
